In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers
from tensorflow.keras import layers, optimizers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l2

2023-12-12 17:42:07.606939: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 17:42:07.973549: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [7]:
# local paths 
training_dir = "train_images_inferred_bad5/train_images" #This is the inferred augmented dir
testing_dir = "test_images" 

In [2]:
# Import data
class_names_dict = np.load("class_names.npy", allow_pickle=True).item()
attributes_npy = np.load("attributes.npy", allow_pickle=True)
attributes_df = pd.read_csv("attributes.txt", header=None, index_col = 0, sep=" |::", names=("attribute", "value"), engine='python')
training_labels_df = pd.read_csv("train_images.csv", header = 0)
test_images_paths = pd.read_csv("test_images_path.csv", header = 0)
test_images_sample = pd.read_csv("test_images_sample.csv", header = 0)

In [3]:
attributes_npy_standarize = np.copy(attributes_npy)

for col_idx in range(attributes_npy.shape[1]):

    current_column = attributes_npy[:, col_idx]

    z_scores = (current_column - np.mean(current_column)) / np.std(current_column)

    threshold = 1
    attributes_npy_standarize[:, col_idx] = np.where(z_scores > threshold, 1, 0)

print(attributes_npy_standarize)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]]


In [27]:
# This loads the training images from the balanced augmented training dir

images = []
labels = []

resize_dim = (224, 224)

# Iterate through subdirectories
for subdir in range(1, 201):
    subdir_path = os.path.join(training_dir, str(subdir))
    counter = 0
    # Iterate through files in each subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith(".jpg"):
            
            # Read images
            image_path = os.path.join(subdir_path, filename)
            img = cv2.imread(image_path)
            
            # Fix images
            img_resized = cv2.resize(img, resize_dim)
            img_normalized = img_resized / 255.0
            
            #Load images
            images.append(img_normalized)
            labels.append(subdir)
            
            #read only 20 images from each subfolder
            #counter += 1
            #if counter >= 20:
            #    break

# Convert lists to numpy arrays
images_array = np.array(images)
labels_array = np.array(labels)


In [28]:
labels_array.shape

(10000,)

In [29]:
attributes_as_labels = []

# Iterate over each label and find its position in the array_of_arrays
for label in labels_array:
    # Subtracting 1 to match the 0-based indexing of arrays
    position = label - 1
    
    # Append the corresponding array to the result_array
    attributes_as_labels.append(attributes_npy_standarize[position])

# result_array now contains 3000 arrays
#print(attributes_as_labels)

#covert to array
attributes_as_labels = np.array(attributes_as_labels)

In [40]:
print(attributes_as_labels.shape)
print(labels_array.shape)
print(images_array.shape)

(10000, 312)
(10000,)
(10000, 224, 224, 3)


## Build Model

In [45]:
# Split to training and val set

train_birds_set, val_birds_set, train_labels_set, val_labels_set = train_test_split(images_array, attributes_as_labels, test_size=0.2, random_state=42)

In [46]:
train_birds_set.shape

(8000, 224, 224, 3)

In [48]:
train_labels_set.shape

(8000, 312)

In [54]:
# Build MODEL

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last 2 
for layer in base_model.layers[-2:]:
    layer.trainable = True


def create_model():
    model = models.Sequential()
    model.add(base_model)
    
    # Add L2 regularization
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(312, activation='softmax', kernel_regularizer=l2(0.001)))
    
    return model

# Create a fresh instance of the model
model = create_model()

In [55]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_birds_set, train_labels_set, epochs=10, validation_data=(val_birds_set, val_labels_set))


Epoch 1/10
250/250 [==============================] - 8s 24ms/step - loss: 0.4947 - accuracy: 0.0426 - val_loss: 0.3673 - val_accuracy: 0.0890
Epoch 2/10
250/250 [==============================] - 5s 20ms/step - loss: 0.3513 - accuracy: 0.0536 - val_loss: 0.3470 - val_accuracy: 0.1060
Epoch 3/10
250/250 [==============================] - 5s 20ms/step - loss: 0.3380 - accuracy: 0.0578 - val_loss: 0.3413 - val_accuracy: 0.0985
Epoch 4/10
250/250 [==============================] - 5s 20ms/step - loss: 0.3310 - accuracy: 0.0514 - val_loss: 0.3372 - val_accuracy: 0.0830
Epoch 5/10
250/250 [==============================] - 5s 20ms/step - loss: 0.3229 - accuracy: 0.0475 - val_loss: 0.3348 - val_accuracy: 0.0265
Epoch 6/10
250/250 [==============================] - 5s 20ms/step - loss: 0.3168 - accuracy: 0.0485 - val_loss: 0.3340 - val_accuracy: 0.0690
Epoch 7/10
250/250 [==============================] - 5s 20ms/step - loss: 0.3107 - accuracy: 0.0430 - val_loss: 0.3336 - val_accuracy: 0.0715